In [1]:
# Pytorch model to do classification. Dataset from folder

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [3]:
# Use pretrained model (Resnet)
model = torchvision.models.resnet34(pretrained=True)

/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/venom/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [4]:
# Input RGB image, normalise to resnet18 and all those stuff
transform = transforms.Compose(
    [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])])

# Load dataset from folder
dataset = torchvision.datasets.ImageFolder(root='/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/augmented_images/', transform=transform)

In [5]:
# Random split dataset into train and test
trainset, testset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])


In [6]:
# Dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                            shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                            shuffle=False, num_workers=2)


In [7]:
# Freeze all layers except last fc layer
for param in model.parameters():
    param.requires_grad = False
    
# Change last fc layer to 7 classes
model.fc = nn.Linear(512, 7)

In [8]:
# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [10]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Train, test in each epoch with tqdm progress bar, use functions for train and test
from tqdm import tqdm

def train():
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print("Training loss: ", running_loss/len(trainloader))

def test():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader, 0)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print("Testing loss: ", running_loss/len(testloader))
    print("Accuracy: ", correct/total)


In [12]:
# Train and test for 20 epochs
for epoch in range(20):
    print("Epoch: ", epoch)
    train()
    test()
    print("\n")



Epoch:  0


525it [00:35, 14.68it/s]

Training loss:  1.5269912078267052



132it [00:08, 14.82it/s]

Testing loss:  1.3739036696426796
Accuracy:  0.4661904761904762


Epoch:  1


...

Testing loss:  1.2662305705475085
Accuracy:  0.5216666666666666


Epoch:  19



525it [00:41, 12.56it/s]

Training loss:  1.1892858566556659



132it [00:10, 12.52it/s]

Testing loss:  1.2244818341551404
Accuracy:  0.5338095238095238




In [13]:
# Unfreeze all layers, and fine tune with smaller learning rate
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=0.00001)


In [14]:
# Train and test for 20 epochs
for epoch in range(20):
    print("Epoch: ", epoch)
    train()
    test()
    print("\n")


Epoch:  0


525it [01:19,  6.62it/s]

Training loss:  0.9829933890842256



132it [00:10, 12.42it/s]

Testing loss:  0.8821205510334535
Accuracy:  0.660952380952381


Epoch:  1


...

Testing loss:  1.1428242867643184
Accuracy:  0.7333333333333333


Epoch:  19



525it [01:20,  6.55it/s]


Training loss:  0.042483609487923485


132it [00:10, 12.14it/s]

Testing loss:  1.148360449256319
Accuracy:  0.7273809523809524




In [15]:
# Make predictions with this model, on test directory, and epxort to csv as per kaggle format
import pandas as pd
import os
from PIL import Image

# Load test directory
test_dir = '/home/venom/repo/Stylumia-Internship-Kaggle/Dataset/test'

# Submission df structure - file_name,label - Lable is predicted class (0-6)
submission_df = pd.DataFrame(columns=['file_name', 'label'])


# Iterate over all files in test directory
for file in tqdm(os.listdir(test_dir)):
    # Open image, resize, convert to tensor, normalise
    img = Image.open(os.path.join(test_dir, file))
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    
    # Get prediction
    output = model(img)
    _, predicted = torch.max(output.data, 1)
    
    # Add to submission df using pandas concat
    submission_df = pd.concat([submission_df, pd.DataFrame([[file, predicted.item()]], columns=['file_name', 'label'])], ignore_index=True)


100%|██████████| 5751/5751 [01:04<00:00, 88.81it/s]


In [17]:
# Export to csv
submission_df.to_csv('ResNet34.csv', index=False)

In [18]:
# Save model
torch.save(model.state_dict(), 'ResNet34.pth')
